# Reshape Accumulations 2016

In [6]:
import sys
import glob
import os
sys.path.insert(0, '../../data/lib/')

import numpy as np
import pandas as pd
import importlib
import re

import pdfexport
importlib.reload(pdfexport)

from pdfexport import *

## Import CSV

In [7]:
df = pd.read_csv('../../data/0. old data/1. transformation/2_accumulation_2016.csv')

## Check Source

In [8]:
df_source = pd.read_excel("sources/company_to_source.xlsx")

In [9]:
df_test = df.copy()
df_test = df_test.merge(df_source, how='left', left_on = 'source', right_on='tra_translation')

df_test[df_test.tra_translation.isna()]

,year,type,label,count,amount,percent,source,tra_translation,tra_name


## Add HCP & HCO

In [10]:
df_data = df.copy()

#Rename travel_accomodation
df_data.loc[df_data.label == 'travel_accomodation', 'label'] = 'travel_accommodation'

# Create empty dataframe
columns = ['type', 'donations_grants', 'sponsorship', 'registration_fees','travel_accommodation', 'fees', 'related_expenses', 'total', 'source']
df_accu = pd.DataFrame(columns=columns)

# Loop data
for index, row in df_data.iterrows():
    items = df_accu[(df_accu.type == row['type']) & (df_accu.source == row['source'])]
    if len(items) == 0:
        new = {
            'type': row['type'],
            'source': row['source']
        }
        df_accu = df_accu.append(new, ignore_index=True)
        
    df_accu.loc[(df_accu.type == row['type']) & (df_accu.source == row['source']), row['label']] = row['amount']

#Calc total
df_accu = df_accu.fillna(0)
df_accu['total'] = df_accu['donations_grants'] + df_accu['sponsorship'] + df_accu['registration_fees'] + df_accu['travel_accommodation'] + df_accu['fees'] + df_accu['related_expenses']

## Add RnD

In [11]:
df_rnd = pd.read_excel("../../data/0. old data/0. raw/2016/rnd.xlsx")
df_rnd['type'] = 'rnd'

#Rename
df_rnd = df_rnd.rename(columns={'name': 'source', 'amount': 'total'})

#concat
df_total = pd.concat([df_accu, df_rnd], sort=False)

#add year
df_total['year'] = 2016

#save
#df_total.to_csv('../../data/0. old data/1. transformation/3_accumulation_2016_reshaped.csv', index=False)